In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install datasets
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 66.4 MB/s 
     |████████████████████████████████| 140 kB 73.0 MB/s 
     |████████████████████████████████| 101 kB 14.8 MB/s 
     |████████████████████████████████| 1.1 MB 57.9 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
     |████████████████████████████████| 127 kB 75.8 MB/s 
     |████████████████████████████████| 144 kB 75.6 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 72.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dep

In [7]:
import numpy as np
import pandas as pd
from datasets import load_dataset
#load the csvs and parse into dataframes with relative length, absolute length, number of questions rel & abs, size of words relative and absolute 

#dataset = load_dataset('csv', data_files={'train':'../data/PANC-train.csv','test':'../data/PANC-test.csv'})
print("loading csv files...")
X_train = pd.read_csv('/content/gdrive/MyDrive/PANC_datasets/PANC-train_add_features_emotion.csv', header=0, encoding='latin1')
X_test = pd.read_csv('/content/gdrive/MyDrive/PANC_datasets/PANC-test_add_features_emotion.csv', header=0, encoding= 'latin1')

#X_train = pd.read_csv("../Datasets/PANC/additionasl_features/PANC-train_add_features_emotion.csv", index_col=0)
#X_test = pd.read_csv("../Datasets/PANC/additionasl_features/PANC-test_add_features_emotion.csv", index_col=0)

loading csv files...


In [8]:
#just for inspection and normalization
# Add a column named 'split', and specify which split each row belongs to.
X_train['split'] = 'train'
X_test['split'] = 'test'

print('{:,} training samples'.format(len(X_train)))
print('{:,} test samples'.format(len(X_test)))

# Confirm the columns match between the two.
assert(set(X_train.columns) == set(X_test.columns))

17,415 training samples
13,159 test samples


In [9]:
# Combine the two into a single dataframe for processing. We'll split them back
# apart later, using the 'split' column we added.
df = pd.concat([X_train, X_test])
#transform to numerical labels
df.loc[:, 'label'] = (df['label'] == 'predator').apply(int)

df['segment'] = df["segment"].map(lambda x: x.replace('[CLS] ', ''))
df

,Unnamed: 0,label,chatName,segment,length_rel,length_abs,num_questions_rel,num_questions_abs,size_of_words_rel,size_of_words_abs,...,bert_nervousness,bert_optimism,bert_pride,bert_realization,bert_relief,bert_remorse,bert_sadness,bert_surprise,bert_neutral,split
0,0,0,9fbb1cac6583f9df43d968c02cfc9eff,Hey hiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii m/f what...,0.277439,565.0,1.000000,3.0,0.280625,139.957143,...,0.002213,0.002550,0.000968,0.007122,0.000705,0.001172,0.004991,0.008361,0.077860,train
1,1,0,afcdee26c7ba50454fe886e80eb16ded,hi hi easter is gay how are u? yea agree im fr...,0.572314,691.0,0.000000,4.0,0.369517,148.843651,...,0.002361,0.065858,0.004021,0.018808,0.002213,0.001050,0.003822,0.002383,0.109519,train
2,2,1,aggresk8er-0,hello how are you hi how are you goo good that...,0.026667,1184.0,0.857143,16.0,0.260687,320.263490,...,0.000863,0.003288,0.002020,0.007620,0.001647,0.001496,0.005680,0.004913,0.017312,train
3,3,1,aggresk8er-1,Hello hi what ya doin? Nothing much just chill...,0.403315,289.0,0.500000,3.0,0.255186,69.497619,...,0.000915,0.004819,0.000722,0.005034,0.000704,0.015193,0.013506,0.002184,0.022046,train
4,4,1,aggresk8er-4,Hello hi How r u doing good what u been doin? ...,0.016981,1051.0,1.000000,9.0,0.248014,221.249451,...,0.000613,0.012490,0.003142,0.004357,0.002666,0.000648,0.001321,0.002366,0.013954,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13154,13154,0,6377d2c072798e7ef0fd22f0769fdbae,15 m hey im 15 f from hey im bobby usa cool im...,0.131579,994.0,0.857143,8.0,0.227453,221.526190,...,0.001099,0.007855,0.001591,0.011055,0.000818,0.000697,0.002752,0.002364,0.075391,test
13155,13155,0,a2378441841247f277c54964860db8c9,hiii hw r u hiiii are you okay ya u too is you...,0.109890,172.0,0.000000,0.0,0.123126,68.850000,...,0.000959,0.002663,0.001041,0.008098,0.000736,0.000540,0.004075,0.000781,0.773637,test
13156,13156,0,497ec625519d6e28dd360be81d01dbbd,Why should interruptions be more problematic f...,0.811240,3067.0,0.333333,5.0,0.808098,283.292332,...,0.002674,0.010463,0.002572,0.024410,0.001482,0.001058,0.005327,0.004805,0.375396,test
13157,13157,0,dfbec532af50f894e0961bb5ef935c4c,boo aaaaaaaaaah :o you scared me Yes I scared ...,0.060071,549.0,0.750000,5.0,0.315303,137.597619,...,0.006449,0.006909,0.005956,0.014781,0.020089,0.002330,0.009367,0.004675,0.026990,test


In [10]:
import torch
class PANCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):

    return tokenizer(examples["segment"], padding="max_length", truncation=True)


# Re-split the dataset.
test_df = df.loc[df.split == 'test']
train_df = df.loc[df.split == 'train']
#tokenized_data_train = tokenize_function(train_df['segment'])
#tokenized_data_test = tokenize_function(test_df['segment'])

new_df_train = train_df[['segment','label']].copy()
new_df_test = test_df[['segment','label']].copy()

train_dataset = Dataset.from_pandas(new_df_train)
test_dataset = Dataset.from_pandas(new_df_test)

tokenized_datasets_train = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets_test = test_dataset.map(tokenize_function, batched=True)







Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Parameter 'function'=<function tokenize_function at 0x7f66d10d9680> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [12]:
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

tokenized_datasets_train = tokenized_datasets_train.remove_columns(["segment"])

tokenized_datasets_train = tokenized_datasets_train.rename_column("label", "labels")

tokenized_datasets_test = tokenized_datasets_test.remove_columns(["segment"])

tokenized_datasets_test = tokenized_datasets_test.rename_column("label", "labels")

tokenized_datasets_test = tokenized_datasets_test.remove_columns(["__index_level_0__"])
tokenized_datasets_train = tokenized_datasets_train.remove_columns(["__index_level_0__"])


tokenized_datasets_train.set_format("torch")
tokenized_datasets_test.set_format("torch")


#small_train_dataset = tokenized_datasets_train.shuffle(seed=42).select(range(1000))

#small_eval_dataset = tokenized_datasets_test.shuffle(seed=42).select(range(1000))

small_train_dataset = tokenized_datasets_train
small_eval_dataset = tokenized_datasets_test


train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)

eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)





Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler


optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3

num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(

    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps

)

In [ ]:
import torch


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):

    for batch in train_dataloader:

        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)

        loss = outputs.loss

        loss.backward()

        optimizer.step()

        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)

  0%|          | 0/6531 [00:00<?, ?it/s]

In [ ]:
model.save_pretrained('/content/gdrive/MyDrive/saved_model/')

In [ ]:
from datasets import load_metric

metric = load_metric("f1")

model.eval()

for batch in eval_dataloader:

    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():

        outputs = model(**batch)

    logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)

    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'f1': 0.9679915582131551}

In [ ]:
model.save_pretrained('/content/gdrive/MyDrive/saved_model/')

Lemme try sth

In [ ]:
from transformers import TextClassificationPipeline

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)
# outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]

In [ ]:
#pipe(test_dataset)
from transformers.pipelines.pt_utils import KeyDataset

for out in tqdm(pipe(KeyDataset(test_dataset, "segment"))):
    print(out)

  0%|          | 0/13159 [00:00<?, ?it/s]

{'label': 'LABEL_0', 'score': 0.9998406171798706}
{'label': 'LABEL_0', 'score': 0.999841570854187}
{'label': 'LABEL_0', 'score': 0.9998414516448975}
{'label': 'LABEL_0', 'score': 0.9998406171798706}
{'label': 'LABEL_0', 'score': 0.9998418092727661}
{'label': 'LABEL_0', 'score': 0.9998406171798706}
{'label': 'LABEL_0', 'score': 0.9998408555984497}
{'label': 'LABEL_0', 'score': 0.9998416900634766}
{'label': 'LABEL_0', 'score': 0.9998406171798706}
{'label': 'LABEL_0', 'score': 0.9998418092727661}


Token indices sequence length is longer than the specified maximum sequence length for this model (592 > 512). Running this sequence through the model will result in indexing errors


{'label': 'LABEL_0', 'score': 0.999841570854187}


RuntimeError: ignored

# Predict probabilities

In [ ]:
# TODO open csv, convert logits to probabilities, and save them. Then I can just pred_2 = read_csv()

In [14]:
import torch
from transformers import AutoModelForSequenceClassification
from google.colab import drive

drive.mount('/content/gdrive')

model = AutoModelForSequenceClassification.from_pretrained('/content/gdrive/MyDrive/saved_model/')

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from datasets import load_metric
import torch.nn.functional as F

metric = load_metric("f1")
recall_metric = load_metric('recall')
precision_metric = load_metric("precision")

model.eval()

all_logits = []
i=0
for batch in eval_dataloader:

    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():

        outputs = model(**batch)

    logits = outputs.logits
    all_logits += logits.cpu().tolist()

    predictions = torch.argmax(logits, dim=-1)

    metric.add_batch(predictions=predictions, references=batch["labels"])
    recall_metric.add_batch(predictions=predictions, references=batch["labels"])
    precision_metric.add_batch(predictions=predictions, references=batch["labels"])

    i+=1

probabilities = F.softmax(torch.Tensor(all_logits), dim=-1).tolist()

metric.compute()
recall_metric.compute()
precision_metric.compute()

In [ ]:
#probabilities = F.softmax(logits, dim=-1)
#np_prob = probabilities.cpu().detach().tolist()

print(probabilities)

[[0.9998406171798706, 0.00015937168791424483], [0.999841570854187, 0.00015845677990000695], [0.9998414516448975, 0.00015849740884732455], [0.9998406171798706, 0.00015938308206386864], [0.9998418092727661, 0.00015810811601113528], [0.9998406171798706, 0.0001593528431840241], [0.9998408555984497, 0.0001591440523043275], [0.9998416900634766, 0.00015827555034775287], [0.9998406171798706, 0.00015939814329613], [0.9998418092727661, 0.00015815426013432443], [0.999841570854187, 0.0001583702105563134], [0.9998424053192139, 0.0001575834467075765], [0.9998410940170288, 0.00015887069457676262], [0.9998416900634766, 0.00015828324831090868], [0.9998406171798706, 0.00015930453082546592], [0.9998449087142944, 0.00015511825040448457], [0.9998413324356079, 0.00015865618479438126], [0.999841570854187, 0.00015840902051422745], [0.9998416900634766, 0.00015826559683773667], [0.9998407363891602, 0.0001592480402905494], [0.9998407363891602, 0.00015919655561447144], [0.9998438358306885, 0.0001561308599775657],

In [ ]:
df = pd.DataFrame(probabilities, columns =['LABEL_1', 'LABEL_2'])
df.to_csv('y_pred_BERT.csv')